# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686584


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:03<00:45,  1.53s/it]

Rendering models:   9%|▉         | 3/32 [00:05<00:55,  1.92s/it]

Rendering models:  12%|█▎        | 4/32 [00:06<00:41,  1.47s/it]

Rendering models:  16%|█▌        | 5/32 [00:06<00:30,  1.14s/it]

...

Rendering models:  91%|█████████ | 29/32 [00:16<00:00,  3.27it/s]

Rendering models:  94%|█████████▍| 30/32 [00:17<00:00,  2.32it/s]

Rendering models:  97%|█████████▋| 31/32 [00:17<00:00,  2.57it/s]

Rendering models: 100%|██████████| 32/32 [00:17<00:00,  2.60it/s]

Almudys                               0.000896
ethanscamander                        0.001187
AlexMercer                            0.000742
Dynamatt                              0.000538
not-logged-in-48a0e5e746c7954388d8    0.001875
not-logged-in-975ec4b89e5ce0be836b    0.000892
not-logged-in-4c2ff289142efbb5f0fb    0.000497
shocko61                              0.000516
kjkavene                              0.000607
kcarte13                              0.000518
not-logged-in-f7e2260ed4a9517a3e4a    0.001468
not-logged-in-3062e04f9aac26d520c7    0.003027
Galaxy-one                            0.000679
MakenzieHoward                        0.016791
lattecuc                              0.005669
sn346808                              0.001012
not-logged-in-1a6b4bed55628d6003a3    0.168897
KrisKeogh                             0.003621
ElisabethB                            0.000498
not-logged-in-120167733761deee9007    0.000708
astronphoenix                         0.000701
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())